## Install Libraries


In [2]:
%pip install "ray[tune]"
# %pip install torch torchvision torchaudio # Uncomment this to install PyTorch 2.0+ on ilab (required version to run this program)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 92.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 KB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 KB 51.9 MB/s eta 0:00:00


## Import Libraries


In [3]:
from filelock import FileLock
from ray import tune
from ray.air import session
from ray.air.checkpoint import Checkpoint
from torch.utils.data import random_split
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from collections import OrderedDict


## Data loaders


In [4]:
def load_data(data_dir="./data"):
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )

    with FileLock(os.path.expanduser("~/.data.lock")):
        trainset = torchvision.datasets.CIFAR10(
            root=data_dir, train=True, download=True, transform=transform
        )

        testset = torchvision.datasets.CIFAR10(
            root=data_dir, train=False, download=True, transform=transform
        )

    return trainset, testset


## Configurable neural network


In [5]:
def flatten(x, start_dim=1, end_dim=-1):
    return x.flatten(start_dim=start_dim, end_dim=end_dim)


In [35]:
# Will need to modify the parameters and the network depending on what you are experimenting with

C, H, W = 3, 32, 32
num_classes = 10

channel_1 = 16
channel_2 = 32
channel_3 = 64
pool_kernel_size = 2

class Net(nn.Module):
    def __init__(self, kernel_size_1, pad_size_1, kernel_size_2, pad_size_2, kernel_size_3, pad_size_3, fc_count_1, fc_count_2, drop):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, channel_1, kernel_size_1, padding=pad_size_1)
        self.norm1 = nn.BatchNorm2d(channel_1)
        self.pool1 = nn.MaxPool2d(pool_kernel_size)
        self.conv2 = nn.Conv2d(channel_1, channel_2, kernel_size_2, padding=pad_size_2)
        self.norm2 = nn.BatchNorm2d(channel_2)
        self.pool2 = nn.MaxPool2d(pool_kernel_size)
        self.conv3 = nn.Conv2d(channel_2, channel_3, kernel_size_3, padding=pad_size_3)
        self.norm3 = nn.BatchNorm2d(channel_3)
        self.pool3 = nn.MaxPool2d(pool_kernel_size)        

        H_after_conv1 = int(H + 2*pad_size_1 - 1*(kernel_size_1-1) - 1 + 1)
        W_after_conv1 = int(W + 2*pad_size_1 - 1*(kernel_size_1-1) - 1 + 1)
        H_after_pool1 = int((H_after_conv1 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        W_after_pool1 = int((W_after_conv1 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        H_after_conv2 = int(H_after_pool1 + 2*pad_size_2 - 1*(kernel_size_2-1) - 1 + 1)
        W_after_conv2 = int(W_after_pool1 + 2*pad_size_2 - 1*(kernel_size_2-1) - 1 + 1)
        H_after_pool2 = int((H_after_conv2 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        W_after_pool2 = int((W_after_conv2 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)  
        H_after_conv3 = int(H_after_pool2 + 2*pad_size_3 - 1*(kernel_size_3-1) - 1 + 1)
        W_after_conv3 = int(W_after_pool2 + 2*pad_size_3 - 1*(kernel_size_3-1) - 1 + 1)
        H_after_pool3 = int((H_after_conv3 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)
        W_after_pool3 = int((W_after_conv3 + 2*0 - 1*(pool_kernel_size-1) - 1) / pool_kernel_size + 1)

        self.fc1 = nn.Linear(H_after_pool3*W_after_pool3*channel_3, fc_count_1)
        if drop == 1:
          self.fc2 = nn.Linear(fc_count_1, fc_count_2)
          self.fc3 = nn.Linear(fc_count_2, 10)
        else:
          self.fc3 = nn.Linear(fc_count_1, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.norm1(x)
        x = F.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.norm2(x)
        x = F.relu(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.norm3(x)
        x = F.relu(x)
        x = self.pool3(x)
        x = flatten(x)
        x = self.fc1(x)
        #x = F.relu(x)
        if hasattr(self, 'drop'):
          x = self.fc2(x)
        #x = F.relu(x)
        x = self.fc3(x)

        return x



## The train function


In [27]:
def train_cifar(config):
    net = Net(
        config["kernel_size_1"], config["pad_size_1"], config["kernel_size_2"], config["pad_size_2"], 
        config["kernel_size_3"], config["pad_size_3"], config["fc_count_1"], config["fc_count_2"], config["drop"] 
    )  # Will need to modify the parameters depending on what you are experimenting with

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=1e-3)

    # Do not modify any code below!
    loaded_checkpoint = session.get_checkpoint()
    if loaded_checkpoint:
        with loaded_checkpoint.as_directory() as loaded_checkpoint_dir:
            model_state, optimizer_state = torch.load(
                os.path.join(loaded_checkpoint_dir, "checkpoint.pt")
            )
            net.load_state_dict(model_state)
            optimizer.load_state_dict(optimizer_state)

    data_dir = os.path.abspath("./data")
    trainset, testset = load_data(data_dir)

    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs]
    )

    trainloader = torch.utils.data.DataLoader(
        train_subset, batch_size=64, shuffle=True, num_workers=8
    )
    valloader = torch.utils.data.DataLoader(
        val_subset, batch_size=64, shuffle=True, num_workers=8
    )

    for epoch in range(10):
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:
                print(
                    f"[{epoch + 1}, {i + 1:>5} loss: {running_loss / epoch_steps:.3f}]"
                )
                running_loss = 0.0

        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        os.makedirs("my_model", exist_ok=True)
        torch.save((net.state_dict(), optimizer.state_dict()), "my_model/checkpoint.pt")
        checkpoint = Checkpoint.from_directory("my_model")
        session.report(
            {"loss": (val_loss / val_steps), "accuracy": correct / total},
            checkpoint=checkpoint,
        )
    print("Finished Training")


## Test set accuracy


In [21]:
def test_best_model(best_result):
    best_trained_model = Net(
        best_result.config["kernel_size_1"], best_result.config["pad_size_1"], best_result.config["kernel_size_2"], best_result.config["pad_size_2"], 
        best_result.config["kernel_size_3"], best_result.config["pad_size_3"], best_result.config["fc_count_1"], best_result.config["fc_count_2"], best_result.config["drop"]  
    )  # Will need to modify the parameters depending on what you are experimenting with

    # Do not modify any code below!
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    best_trained_model.to(device)

    checkpoint_path = os.path.join(
        best_result.checkpoint.to_directory(), "checkpoint.pt"
    )

    model_state, optimizer_state = torch.load(checkpoint_path)
    best_trained_model.load_state_dict(model_state)

    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2
    )

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = best_trained_model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Best trial test set accuracy: {correct / total}")


## Configuring the search space


In [36]:
# You will be experimenting with the hyperparameters here
# Use tune.grid_search to specify what values you want to experiment with a hyperparameter
config = {"kernel_size_1": tune.grid_search([3, 5]), 
          "pad_size_1": tune.grid_search([1, 2]), 
          "kernel_size_2": tune.grid_search([3, 5]), 
          "pad_size_2": tune.grid_search([1, 2]), 
          "kernel_size_3": tune.grid_search([5, 7]), 
          "pad_size_3": tune.grid_search([2, 3]), 
          "fc_count_1": tune.grid_search([512]),
          "fc_count_2": tune.grid_search([512]),
          "drop": tune.grid_search([0, 1])}



In [37]:
# Do not modify any code below!
def main(config):
    tuner = tune.Tuner(
        tune.with_resources(
            tune.with_parameters(train_cifar),
            resources={"gpu": 1},
        ),
        param_space=config,
    )
    results = tuner.fit()

    best_result = results.get_best_result("accuracy", "max")

    print(f"Best trial config: {best_result.config}")
    print(f"Best trial final validation loss: {best_result.metrics['loss']}")
    print(f"Best trial final validation accuracy: {best_result.metrics['accuracy']}")

    test_best_model(best_result)

    return results.get_dataframe("accuracy", "max")


results_df = main(config)


(train_cifar pid=310708) Downloading https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-08_03-23-24/train_cifar_b852e_00000_0_drop=0,fc_count_1=512,fc_count_2=512,kernel_size_1=3,kernel_size_2=3,kernel_size_3=5,pad_size_1=1,pad_si_2023-04-08_03-23-24/data/cifar-10-python.tar.gz


100%|██████████| 170498071/170498071 [00:05<00:00, 28956811.27it/s]


(train_cifar pid=310708) Extracting /root/ray_results/train_cifar_2023-04-08_03-23-24/train_cifar_b852e_00000_0_drop=0,fc_count_1=512,fc_count_2=512,kernel_size_1=3,kernel_size_2=3,kernel_size_3=5,pad_size_1=1,pad_si_2023-04-08_03-23-24/data/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-08_03-23-24/train_cifar_b852e_00000_0_drop=0,fc_count_1=512,fc_count_2=512,kernel_size_1=3,kernel_size_2=3,kernel_size_3=5,pad_size_1=1,pad_si_2023-04-08_03-23-24/data
(train_cifar pid=310708) Files already downloaded and verified


Trial name,accuracy,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,loss,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_cifar_b852e_00000,0.6081,2023-04-08_03-24-25,True,,1811d02932b44de69a2395be01c9762e,"0_drop=0,fc_count_1=512,fc_count_2=512,kernel_size_1=3,kernel_size_2=3,kernel_size_3=5,pad_size_1=1,pad_size_2=1,pad_size_3=2",9ca3bf134adb,10,1.1199,172.28.0.12,310708,True,58.3541,4.37479,58.3541,1680924265,0,,10,b852e_00000,0.00354028
train_cifar_b852e_00001,0.5941,2023-04-08_03-25-19,True,,1811d02932b44de69a2395be01c9762e,"1_drop=1,fc_count_1=512,fc_count_2=512,kernel_size_1=3,kernel_size_2=3,kernel_size_3=5,pad_size_1=1,pad_size_2=1,pad_size_3=2",9ca3bf134adb,10,1.15506,172.28.0.12,310708,True,53.8949,4.37335,53.8949,1680924319,0,,10,b852e_00001,0.00354028
train_cifar_b852e_00002,0.5669,2023-04-08_03-26-14,True,,1811d02932b44de69a2395be01c9762e,"2_drop=0,fc_count_1=512,fc_count_2=512,kernel_size_1=5,kernel_size_2=3,kernel_size_3=5,pad_size_1=1,pad_size_2=1,pad_size_3=2",9ca3bf134adb,10,1.22742,172.28.0.12,310708,True,54.3061,4.58977,54.3061,1680924374,0,,10,b852e_00002,0.00354028
train_cifar_b852e_00003,0.5996,2023-04-08_03-27-08,True,,1811d02932b44de69a2395be01c9762e,"3_drop=1,fc_count_1=512,fc_count_2=512,kernel_size_1=5,kernel_size_2=3,kernel_size_3=5,pad_size_1=1,pad_size_2=1,pad_size_3=2",9ca3bf134adb,10,1.15442,172.28.0.12,310708,True,54.2082,4.58259,54.2082,1680924428,0,,10,b852e_00003,0.00354028
train_cifar_b852e_00004,0.5858,2023-04-08_03-28-01,True,,1811d02932b44de69a2395be01c9762e,"4_drop=0,fc_count_1=512,fc_count_2=512,kernel_size_1=3,kernel_size_2=5,kernel_size_3=5,pad_size_1=1,pad_size_2=1,pad_size_3=2",9ca3bf134adb,10,1.17979,172.28.0.12,310708,True,53.6152,4.33404,53.6152,1680924481,0,,10,b852e_00004,0.00354028
train_cifar_b852e_00005,0.5961,2023-04-08_03-28-55,True,,1811d02932b44de69a2395be01c9762e,"5_drop=1,fc_count_1=512,fc_count_2=512,kernel_size_1=3,kernel_size_2=5,kernel_size_3=5,pad_size_1=1,pad_size_2=1,pad_size_3=2",9ca3bf134adb,10,1.15195,172.28.0.12,310708,True,53.5861,4.42996,53.5861,1680924535,0,,10,b852e_00005,0.00354028
train_cifar_b852e_00006,0.585,2023-04-08_03-29-50,True,,1811d02932b44de69a2395be01c9762e,"6_drop=0,fc_count_1=512,fc_count_2=512,kernel_size_1=5,kernel_size_2=5,kernel_size_3=5,pad_size_1=1,pad_size_2=1,pad_size_3=2",9ca3bf134adb,10,1.19761,172.28.0.12,310708,True,54.6958,4.57044,54.6958,1680924590,0,,10,b852e_00006,0.00354028
train_cifar_b852e_00007,0.5908,2023-04-08_03-30-44,True,,1811d02932b44de69a2395be01c9762e,"7_drop=1,fc_count_1=512,fc_count_2=512,kernel_size_1=5,kernel_size_2=5,kernel_size_3=5,pad_size_1=1,pad_size_2=1,pad_size_3=2",9ca3bf134adb,10,1.16963,172.28.0.12,310708,True,54.2094,4.25246,54.2094,1680924644,0,,10,b852e_00007,0.00354028
train_cifar_b852e_00008,0.6218,2023-04-08_03-31-38,True,,1811d02932b44de69a2395be01c9762e,"8_drop=0,fc_count_1=512,fc_count_2=512,kernel_size_1=3,kernel_size_2=3,kernel_size_3=7,pad_size_1=1,pad_size_2=1,pad_size_3=2",9ca3bf134adb,10,1.0875,172.28.0.12,310708,True,54.1862,4.41643,54.1862,1680924698,0,,10,b852e_00008,0.00354028
train_cifar_b852e_00009,0.6043,2023-04-08_03-32-32,True,,1811d02932b44de69a2395be01c9762e,"9_drop=1,fc_count_1=512,fc_count_2=512,kernel_size_1=3,kernel_size_2=3,kernel_size_3=7,pad_size_1=1,pad_size_2=1,pad_size_3=2",9ca3bf134adb,10,1.13778,172.28.0.12,310708,True,54.2226,4.4812,54.2226,1680924752,0,,10,b852e_00009,0.00354028


(train_cifar pid=310708) Finished Training
(train_cifar pid=310708) Downloading https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-08_03-23-24/train_cifar_b852e_00001_1_drop=1,fc_count_1=512,fc_count_2=512,kernel_size_1=3,kernel_size_2=3,kernel_size_3=5,pad_size_1=1,pad_si_2023-04-08_03-24-25/data/cifar-10-python.tar.gz


...

(train_cifar pid=310708) Extracting /root/ray_results/train_cifar_2023-04-08_03-23-24/train_cifar_b852e_00126_126_drop=0,fc_count_1=512,fc_count_2=512,kernel_size_1=5,kernel_size_2=5,kernel_size_3=7,pad_size_1=2,pad__2023-04-08_05-19-28/data/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-08_03-23-24/train_cifar_b852e_00126_126_drop=0,fc_count_1=512,fc_count_2=512,kernel_size_1=5,kernel_size_2=5,kernel_size_3=7,pad_size_1=2,pad__2023-04-08_05-19-28/data
(train_cifar pid=310708) Files already downloaded and verified
(train_cifar pid=310708) Finished Training
(train_cifar pid=310708) Downloading https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-08_03-23-24/train_cifar_b852e_00127_127_drop=1,fc_count_1=512,fc_count_2=512,kernel_size_1=5,kernel_size_2=5,kernel_size_3=7,pad_size_1=2,pad__2023-04-08_05-20-24/data/cifar-10-python.tar.gz


100%|██████████| 170498071/170498071 [00:05<00:00, 29620304.93it/s]


(train_cifar pid=310708) Extracting /root/ray_results/train_cifar_2023-04-08_03-23-24/train_cifar_b852e_00127_127_drop=1,fc_count_1=512,fc_count_2=512,kernel_size_1=5,kernel_size_2=5,kernel_size_3=7,pad_size_1=2,pad__2023-04-08_05-20-24/data/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2023-04-08_03-23-24/train_cifar_b852e_00127_127_drop=1,fc_count_1=512,fc_count_2=512,kernel_size_1=5,kernel_size_2=5,kernel_size_3=7,pad_size_1=2,pad__2023-04-08_05-20-24/data
(train_cifar pid=310708) Files already downloaded and verified
(train_cifar pid=310708) Finished Training


2023-04-08 05:21:20,622	INFO tune.py:798 -- Total run time: 7076.49 seconds (7076.42 seconds for the tuning loop).


Best trial config: {'kernel_size_1': 5, 'pad_size_1': 2, 'kernel_size_2': 3, 'pad_size_2': 2, 'kernel_size_3': 7, 'pad_size_3': 3, 'fc_count_1': 512, 'fc_count_2': 512, 'drop': 1}
Best trial final validation loss: 1.0804343477935547
Best trial final validation accuracy: 0.6283
Files already downloaded and verified
Files already downloaded and verified
Best trial test set accuracy: 0.5803


In [39]:
final_df = results_df.drop(
    columns=[
        "should_checkpoint",
        "timesteps_total",
        "episodes_total",
        "training_iteration",
        "trial_id",
        "experiment_id",
        "date",
        "timestamp",
        "pid",
        "hostname",
        "node_ip",
        "time_since_restore",
        "timesteps_since_restore",
        "iterations_since_restore",
        "warmup_time",
        "logdir",
    ]
)
final_df.sort_values(by=['accuracy'], ascending=False).head(10)

,loss,accuracy,time_this_iter_s,done,time_total_s,config/drop,config/fc_count_1,config/fc_count_2,config/kernel_size_1,config/kernel_size_2,config/kernel_size_3,config/pad_size_1,config/pad_size_2,config/pad_size_3
123,1.080434,0.6283,4.457087,False,55.751172,1,512,512,5,3,7,2,2,3
118,1.066620,0.6279,4.547004,False,55.285260,0,512,512,5,5,5,2,2,3
61,1.060227,0.6260,4.596972,False,55.365832,1,512,512,3,5,7,2,2,2
100,1.075836,0.6256,4.590197,False,55.508397,0,512,512,3,5,5,1,2,3
89,1.088328,0.6245,4.638068,False,55.074173,1,512,512,3,3,7,2,1,3
60,1.087312,0.6223,4.602597,False,56.561031,0,512,512,3,5,7,2,2,2
8,1.087505,0.6218,4.416425,False,54.186180,0,512,512,3,3,7,1,1,2
116,1.099848,0.6214,4.781852,False,55.710179,0,512,512,3,5,5,2,2,3
125,1.076874,0.6213,4.588823,False,55.819689,1,512,512,3,5,7,2,2,3
45,1.093875,0.6210,4.575586,False,55.423639,1,512,512,3,5,7,1,2,2


In [40]:
final_df.sort_values(by=['accuracy']).head(10)

,loss,accuracy,time_this_iter_s,done,time_total_s,config/drop,config/fc_count_1,config/fc_count_2,config/kernel_size_1,config/kernel_size_2,config/kernel_size_3,config/pad_size_1,config/pad_size_2,config/pad_size_3
2,1.227424,0.5669,4.589772,False,54.306092,0,512,512,5,3,5,1,1,2
66,1.219620,0.5731,4.444452,False,50.734903,0,512,512,5,3,5,1,1,3
39,1.204690,0.5740,4.423295,False,54.385145,1,512,512,5,5,5,1,2,2
22,1.169691,0.5801,4.555981,False,54.735014,0,512,512,5,5,5,2,1,2
15,1.176450,0.5820,4.376045,False,53.898010,1,512,512,5,5,7,1,1,2
6,1.197611,0.5850,4.570438,False,54.695825,0,512,512,5,5,5,1,1,2
34,1.189117,0.5852,4.377496,False,54.717328,0,512,512,5,3,5,1,2,2
49,1.181759,0.5854,4.732704,False,55.016229,1,512,512,3,3,5,2,2,2
98,1.172074,0.5857,4.682020,False,55.696247,0,512,512,5,3,5,1,2,3
4,1.179792,0.5858,4.334037,False,53.615185,0,512,512,3,5,5,1,1,2
